In [1]:
import numpy as np 
import pandas as pd

In [2]:
from utils import *
from preprocessing import *

In [3]:
# load data using utf8 format

data = pd.read_csv('./datasets/combined_datasets.csv', encoding='utf-8')

In [4]:
# fill 0 to num_like_post, num_commnet_post, num_share_post
data.num_like_post.fillna(0, inplace=True)
data.num_comment_post.fillna(0, inplace=True)
data.num_share_post.fillna(0, inplace=True)
data = data.dropna(subset=['post_message'], how='any').reset_index(drop=True)

# check again
check_like = data.num_like_post.isna().sum()
check_commnet = data.num_comment_post.isna().sum()
check_share = data.num_share_post.isna().sum()
check_post_message = data.post_message.isna().sum()

print(f'num_like_post has nan {check_like != 0}')
print(f'num_comment_post has nan {check_commnet != 0}')
print(f'num_share_post has nan {check_share != 0}')
print(f'post_message has nan {check_post_message != 0}')

num_like_post has nan False
num_comment_post has nan False
num_share_post has nan False
post_message has nan False


In [5]:
# drop id, user_name, timestamp_post columns
data = data.drop(['timestamp_post', 'Unnamed: 0'], axis=1).reset_index(drop=True)

# label encoding for user_name
from sklearn.preprocessing import LabelEncoder
labels = data.user_name.values

le = LabelEncoder()
data['user_name_labelEncoder'] = pd.Series(le.fit_transform(labels))

# drop duplicate samples has <URL>, [<URL>](<URL>)
char_drop = ['<URL>', '[<URL>](<URL>)']
index = []
for char in char_drop:
    index.extend(list(data[data.post_message == char].index))
data = data.drop(index, axis=0).reset_index(drop=True)

# view data
data.head(5)

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0,1032
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0,2804
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0,2044
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0,3575
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701


In [6]:
count = data.user_name.value_counts()
keys = count.index.values.tolist()
values = count.values.tolist()

keys_values = zip(keys, values)
lookup = dict(keys_values)

user_name_freq = [lookup[x] for x in data.user_name.values]
data['user_name_freq'] = pd.Series(user_name_freq)

data.head(5)

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0,1032,58
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0,2804,1
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0,2044,64
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0,3575,1
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701,1


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5111 entries, 0 to 5110
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      5111 non-null   int64 
 1   user_name               5111 non-null   object
 2   post_message            5111 non-null   object
 3   num_like_post           5111 non-null   object
 4   num_comment_post        5111 non-null   object
 5   num_share_post          5111 non-null   object
 6   label                   5111 non-null   int64 
 7   user_name_labelEncoder  5111 non-null   int32 
 8   user_name_freq          5111 non-null   int64 
dtypes: int32(1), int64(3), object(5)
memory usage: 339.5+ KB


In [8]:
index = []
for i in range(len(data)):
    try:
        int(float(data.num_comment_post[i]))
        int(float(data.num_like_post[i]))
        int(float(data.num_share_post[i]))
    except:
        index.append(i)
data_temp = data.iloc[index,:]
data_temp

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq
34,36,fe8176d5bc081419f9b08e195525c131,Người mặt đen miệng ngậm 2 quả gì đó tay cầm đ...,unknown,unknown,unknown,1,3687,1
76,78,45d80e8d62eac8e3f28a8c822fab002d,Mọi người ra đường nhớ đeo khẩu trang nha. Đã ...,unknown,unknown,unknown,1,1210,1
80,82,f8584733a5e8e99a9b45ebb5400e796e,MU nhuộm đỏ thành Manchester,Solskjaer hạ knock-out Guardiola,Juventus bỏ xa Inter Milan... là những ảnh ch...,1583713929.0,0,3582,1
174,178,0adfdeb881147078b0c50354193d67dd,Ca Phẫu Thuật thất bại. Hoài Linh lâm vào tình...,unknown,unknown,unknown,1,340,1
185,190,a2ed99660efc25ebf7dd8947c046fd4c,Hà Nội đã hoãn giải đua F1 do dịch Covid-19,29,1 comment,0,0,2518,1
...,...,...,...,...,...,...,...,...,...
4494,183,3498669648858314752,Vừa cập nhật dc 13 học sinh lớp 1 trường Hương...,unknown,unknown,unknown,1,977,1
4496,185,3380197056197186048,Sợ thật. Dịch covid đã về đến Hưng Nguyên và N...,unknown,unknown,unknown,1,957,1
4497,186,-6852763066036545536,BÀI THUỐC MỖI SÁNG KHIẾN TÔI KHỎE MẠNH\n\nKhuy...,10,5,1 share,1,163,1
4509,198,-6478801688689956864,VTV1 CŨNG ĐÃ NÓI RỒI NÊN MỌI NGƯỜI CỨ LÀM NHA....,unknown,unknown,unknown,1,152,1


In [9]:
index_drop = []
for i in range(len(data)):
    try:
        int(float(data.num_comment_post[i]))
        int(float(data.num_like_post[i]))
        int(float(data.num_share_post[i]))
    except:
        if int(float(data.label[i])) == 0:
            index_drop.append(i)
        else:
            data.num_comment_post[i] = 0
            data.num_like_post[i] = 0
            data.num_share_post[i] = 0

data = data.drop(index=index_drop, axis=0).reset_index(drop=True)
data

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173.0,0,1032,58
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19.0,0,2804,1
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0.0,0,2044,64
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54.0,0,3575,1
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701,1
...,...,...,...,...,...,...,...,...,...
5095,795,2490795936720330752,"Định cư gần 60 năm, trong đó 20 năm bị ""treo"" ...",0,0,0,0,757,1
5096,796,1846872478075934976,"Loạn não quá\n\nTheo báo điện tử Giao thông, p...",43,24,3,0,583,2
5097,797,-4723502351165939712,https://m.baophapluat.vn/ban-doc/vu-an-tranh-c...,3,0,0,0,94,2
5098,798,7717037201428829184,"Cục Bảo vệ thực vật cho rằng, đề xuất tạm thời...",13,2,0,0,1910,7


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      5100 non-null   int64 
 1   user_name               5100 non-null   object
 2   post_message            5100 non-null   object
 3   num_like_post           5100 non-null   object
 4   num_comment_post        5100 non-null   object
 5   num_share_post          5100 non-null   object
 6   label                   5100 non-null   int64 
 7   user_name_labelEncoder  5100 non-null   int32 
 8   user_name_freq          5100 non-null   int64 
dtypes: int32(1), int64(3), object(5)
memory usage: 338.8+ KB


In [11]:
# convert post_message, num_like_post, num_comment_post, num_share_post to int64
columns = ['num_like_post', 'num_comment_post', 'num_share_post', 'label', 'user_name_labelEncoder', 'user_name_freq']

data[columns] = data[columns].apply(pd.to_numeric).astype('int64')

data

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173,0,1032,58
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19,0,2804,1
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0,0,2044,64
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54,0,3575,1
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701,1
...,...,...,...,...,...,...,...,...,...
5095,795,2490795936720330752,"Định cư gần 60 năm, trong đó 20 năm bị ""treo"" ...",0,0,0,0,757,1
5096,796,1846872478075934976,"Loạn não quá\n\nTheo báo điện tử Giao thông, p...",43,24,3,0,583,2
5097,797,-4723502351165939712,https://m.baophapluat.vn/ban-doc/vu-an-tranh-c...,3,0,0,0,94,2
5098,798,7717037201428829184,"Cục Bảo vệ thực vật cho rằng, đề xuất tạm thời...",13,2,0,0,1910,7


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5100 entries, 0 to 5099
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      5100 non-null   int64 
 1   user_name               5100 non-null   object
 2   post_message            5100 non-null   object
 3   num_like_post           5100 non-null   int64 
 4   num_comment_post        5100 non-null   int64 
 5   num_share_post          5100 non-null   int64 
 6   label                   5100 non-null   int64 
 7   user_name_labelEncoder  5100 non-null   int64 
 8   user_name_freq          5100 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 358.7+ KB


In [13]:
# preprocessing post_message uses stopword
from vncorenlp import VnCoreNLP

stopwords_file = "vietnamese-stopwords/vietnamese-stopwords.txt"

stopwords = get_stop_words(stopwords_file)

annotator = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

data['post_message_preproced'] = data.post_message.apply(text_preprocessing, stopwords= stopwords, annotator= annotator)

data.head(5)

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq,post_message_preproced
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173,0,1032,58,thăng cấp_bậc hàm cán_bộ chiến_sỹ hy_sinh đà_n...
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19,0,2804,1,tư_vấn mùa thi nộp hồ_sơ trúng_tuyển chương_tr...
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0,0,2044,64,cơ_quan cạnh_tranh thị_trường quyết_định điều_...
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54,0,3575,1,ca quảng_nam hành_khách chuyến bay bệnh_nhân n...
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701,1,học thể dụ c thầy_giáo nguyễn_văn quân phụ trá...


In [14]:
# preprocessing post_message don't eliminate stopword
from vncorenlp import VnCoreNLP

stopwords_file = "vietnamese-stopwords/custom-vietnamese-stopwords.txt"

stopwords = get_stop_words(stopwords_file)

annotator = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

data['post_message_notStopword'] = data.post_message.apply(text_preprocessing, stopwords= stopwords, annotator= annotator)

data.head(5)

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq,post_message_preproced,post_message_notStopword
0,1,389c669730cb6c54314a46be785cea42,"THĂNG CẤP BẬC HÀM ĐỐI VỚI 2 CÁN BỘ, CHIẾN SỸ H...",19477,378,173,0,1032,58,thăng cấp_bậc hàm cán_bộ chiến_sỹ hy_sinh đà_n...,thăng cấp_bậc hàm đối_với cán_bộ chiến_sỹ hy_s...
1,3,b9f3394d2aff86d85974f5040c401f08,TƯ VẤN MÙA THI: Cách nộp hồ sơ để trúng tuyển ...,48,5,19,0,2804,1,tư_vấn mùa thi nộp hồ_sơ trúng_tuyển chương_tr...,tư_vấn mùa thi cách nộp hồ_sơ để trúng_tuyển c...
2,4,808e278b22ec6b96f2faf7447d10cd8e,Cơ quan Cạnh tranh và Thị trường Anh quyết địn...,3,0,0,0,2044,64,cơ_quan cạnh_tranh thị_trường quyết_định điều_...,cơ_quan cạnh_tranh và thị_trường anh quyết_địn...
3,5,f81bdd6d8be4c5f64bb664214e47aced,Thêm 7 ca tại Quảng Nam liên quan đến hành khá...,775,0,54,0,3575,1,ca quảng_nam hành_khách chuyến bay bệnh_nhân n...,thêm ca tại quảng_nam liên_quan đến hành_khách...
4,6,ffc4b6bab27c40cfc48e4dc8b8a41e42,Trong giờ học Thể dụ‌c do thầy giáo Nguyễn Văn...,2,1,0,0,3701,1,học thể dụ c thầy_giáo nguyễn_văn quân phụ trá...,trong giờ học thể dụ c do thầy_giáo nguyễn_văn...


In [15]:
# split train, test dataset
size = int(0.8*len(data))
ids_train = np.random.choice(len(data), replace=False, size=size) # ids of train
train = data.iloc[ids_train]

ids_test = [x not in ids_train for x in range(len(data))]
test = data.iloc[ids_test]

print('Size of train set:', len(train))
print('Size of test set:', len(test))

Size of train set: 4080
Size of test set: 1020


In [16]:
train.to_csv('train.csv', encoding='utf8', index=False)
test.to_csv('test.csv', encoding='utf8', index=False)

In [17]:
data = pd.read_csv('train.csv', encoding='utf8')

In [18]:
# down samples
index_0 = list(data[data.label == 0].index)
index_1 = list(data[data.label == 1].index)

ids = list(np.random.choice(index_0, size=int(1.1*len(index_1)), replace=False))

index = ids + index_1

data.iloc[index]

,id,user_name,post_message,num_like_post,num_comment_post,num_share_post,label,user_name_labelEncoder,user_name_freq,post_message_preproced,post_message_notStopword
261,502,7050444021058475008,"""Ghen Cô Vy"" là 1 dự án sáng tạo của Viện Sức ...",22000,1000,5500,0,1804,28,ghen vy dự_án sáng_tạo viện sức_khoẻ nghề_nghi...,ghen cô vy là dự_án sáng_tạo của viện sức_khoẻ...
1766,1613,ab603852d291d90dde0b1b1a2fd09a2a,"Hnay, Hội đồng thẩm phán sẽ ra phán quyết đối ...",18,3,1,0,2638,1,hnay hội_đồng thẩm_phán phán_quyết tử_tù hồ hả...,hnay hội_đồng thẩm_phán sẽ ra phán_quyết đối_v...
220,913,6f0a57960e94a6489a552226c4bf05f1,"🧪🧪🧪 23h đêm 19/4, CDC Hà Nội thông báo:\n\nKết...",9289,50,184,0,1783,1,emoji đêm cdc hà_nội thông_báo kết_quả trường_...,emoji đêm cdc hà_nội thông_báo kết_quả trường_...
3422,1718,dba9802a1bc2c0c07b64c28dc8e99dba,Hà Nội chi 114 tỷ phun nước hạ nhiệt: Nhiều qu...,0,0,0,0,3212,1,hà_nội chi tỷ phun hạ nhiệt quận_huyện hẹn mùa...,hà_nội chi tỷ phun nước hạ nhiệt nhiều quận_hu...
3798,919,8cec85073924e94f5441a22aeebd68df,"Đối với người dùng cá nhân, tài khoản trên Cổn...",6,0,3,0,2238,1,tài_khoản cổng dịch_vụ công_quốc_gia đăng_ký t...,đối_với người dùng cá_nhân tài_khoản trên cổng...
...,...,...,...,...,...,...,...,...,...,...,...
4057,2906,353409ae5197e7441fe99c567b9f6851,Virus Corona và Lòng Kiêu Ngạo Trung Cộng Tại ...,3100,901,3200,1,985,1,virus corona kiêu_ngạo trung cộng virus corona...,virus corona và lòng kiêu_ngạo trung cộng tại_...
4058,250,bb9d7a3eebe8a9e1d765c6ab01fae5c2,Ai mới là người đáng thương? -----------------...,567,262,377,1,2833,1,thương đọc mấy bài_chỉ trích dân mạng tội chửi...,ai mới là người đáng thương đọc mấy bài_chỉ tr...
4060,3467,f97a7ba6cbfa436206142955a1a2b621,Coppy.\n CỨ VỮNG BỀN MÀ TIẾN TỚI!\n\n Thái Lan...,63,145,2,1,3599,1,copy vững_bền tiến thái_lan sản_xuất triệu xe ...,copy cứ vững_bền mà tiến tới thái_lan một năm ...
4061,2727,bd612a8c9a208033cd7344cba730800a,"Tin chờ xác nhận: ""Giao lưu Quốc phòng"" biên g...",0,0,0,1,2851,1,chờ xác_nhận giao_lưu quốc_phòng biên_giới tây...,tin chờ xác_nhận giao_lưu quốc_phòng biên_giới...


In [19]:
data.iloc[index].to_csv('data_balance.csv', encoding='utf-8', index=False)